# Visualize Notebook
In this notebook the Gold Dataframe will be read and extract information of it. The objective is to see the correlations between the variables and the GDP and also what countries have the highest correlation value.


## Imports
Start importing all the libraries and also the methods of pvalue and search indicators that will be used later in the notebook.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import shapiro
import os
import statistics
import seaborn as sns
from scipy.stats import norm

import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import interact, interact_manual
from Project.Utils.visualize import  search


## Correlation dataframe.
This dataframe is the main piece of the notebook. Consists in generating for every country the correlation matrix for it and saving only the correlation value of the different variables with the GDP. Later on is concatenated and generates the following result:

In [2]:
#One dataframe per country

df= pd.read_csv (os.getcwd()+'/Output/GoldDataframe.csv')
corr_df = pd.DataFrame()
corr_df.index.names = ['Country']
aux_df = pd.DataFrame()

#List all the countries, none repeated
countries = set(df['Country'].to_list())

country_dict = {}
corr_dict = {}

for country in countries:

    #Get the DataFrame for a given country
    country_df = df.loc[df['Country'] == country]

    #Correlation matrix for that country
    country_corr_df = country_df.corr()

    #Trim it into a single row
    country_corr_df = country_corr_df.rename(columns = {'GDP': country}).drop(index = ['Year', 'GDP'])

    #Add the row to a new DataFrame with the correlations for each country
    corr_df = pd.concat([corr_df, country_corr_df[country]], axis = 1)

#Transpose the resulting DataFrame to have the desired format and show it
corr_df = corr_df.transpose()
corr_df


,AgriShareGDP,CreditToAgriFishForest,EmploymentRural,TotalAgri,% Soldiers,Birth Rate,Death Rate,Homicides,Life Expectancy,Maternal Death Risk,...,% Population Growth,% Rural Population,Civil Liberties,Freedom of Expression,% Healthcare Investment,% Employment Industry,% Education Expenditure,% Men Employment,% Women Employment,Population
Switzerland,-0.926746,0.899981,NaN,0.938234,-0.314830,0.271648,-0.875918,-0.917309,0.966205,-0.900268,...,0.576820,-0.933934,-0.138955,NaN,0.947935,-0.856593,-0.049844,-0.920018,0.808883,0.936869
Malaysia,-0.057994,0.055144,NaN,0.860665,-0.944179,-0.854599,0.852503,NaN,0.961953,-0.819080,...,-0.964197,-0.970563,0.619590,0.613580,0.888805,-0.878161,-0.379667,NaN,NaN,0.967304
Belize,-0.606433,0.585657,NaN,0.929205,-0.467308,-0.966932,-0.802729,0.729987,0.971239,-0.962953,...,-0.946497,-0.489393,NaN,NaN,0.812707,-0.475797,NaN,NaN,NaN,0.966144
San Marino,0.065850,NaN,NaN,NaN,NaN,0.357578,-0.162883,NaN,NaN,NaN,...,0.333934,0.001470,NaN,NaN,-0.198150,NaN,NaN,NaN,NaN,-0.004386
Tuvalu,0.200130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.133886,-0.978989,NaN,NaN,0.298903,NaN,NaN,NaN,NaN,0.984535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yemen,0.077711,0.277442,NaN,0.675176,-0.305362,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.026794,0.080743,-0.784325,0.837351,NaN,NaN,NaN,0.595893
Mauritius,-0.968061,NaN,NaN,-0.214700,0.825267,-0.966671,0.820455,-0.322001,0.949731,-0.757616,...,-0.936105,0.961043,-0.499603,0.783910,0.740666,-0.875859,0.490636,NaN,NaN,0.951379
Philippines,-0.899149,-0.332771,NaN,0.938213,-0.937819,-0.988189,0.986778,0.194614,0.993256,-0.989782,...,-0.927512,-0.653935,-0.251800,-0.629247,0.789348,0.360977,NaN,0.120312,0.733025,0.987770
Slovakia,0.546179,NaN,NaN,0.823912,-0.638510,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.466675,-0.347796,0.625439,-0.476850,0.053395,0.653098,0.582675,0.652635


In [3]:
import ipywidgets as widgets
import plotly.express as px


indicator = widgets.SelectMultiple(
    options = corr_df.columns.tolist(),
    value = ['AgriShareGDP'],
    description='Indicator',
    disabled=False,
    layout = Layout(width='50%', height='80px')
)


def globalGrapgh(indicator):
    fig = px.choropleth(corr_df, locations = corr_df.index, locationmode='country names', 
                        color= indicator[0],projection="natural earth",
                        color_continuous_scale='Reds',
                    width=700, height=500)
    fig.update(layout_coloraxis_showscale=True)
    fig.show()
    
    
widgets.interactive(globalGrapgh, indicator = indicator) 

interactive(children=(SelectMultiple(description='Indicator', index=(0,), layout=Layout(height='80px', width='…

In [4]:
def tableOut(Threshold, Country):

    df = search(Threshold, Country, 'Country')
    if df.empty:
        return print("No indicators have been found.")

    left = pd.Series([0.05, 0.05], index=['P-value Pearson', 'P-value Spearman'])
    left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    df =df.style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:red;')\
        .highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\
        .format('{:,.12f}', subset = ['P-value Pearson', 'P-value Spearman']) 
    
    display(df)

    

@interact(
    Country = sorted(corr_df.index.tolist()),
    Threshold = (0, 1, 0.05))
def g(Country = 'Afghanistan', Threshold = 0.7):
    return tableOut(Threshold,Country)

    
        

interactive(children=(Dropdown(description='Country', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra',…

In [5]:
def tableOut(Threshold, Region):

    df = search(Threshold, Region, 'Region')
    if df.empty:
        return print("No indicators have been found.")

    left = pd.Series([0.05, 0.05], index=['P-value Pearson', 'P-value Spearman'])
    left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    df =df.style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:red;')\
        .highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\
        .format('{:,.12f}', subset = ['P-value Pearson', 'P-value Spearman']) 
    
    display(df)

    

@interact(
    Region = set(df['Region'].to_list()),
    Threshold = (0, 1, 0.05))
def g(Region = 'Afghanistan', Threshold = 0.7):
    return tableOut(Threshold, Region)

    

interactive(children=(Dropdown(description='Region', options=('North America', 'Middle East and North Africa',…